## Construction of hstorical database

In [ ]:
# construct_temp_prcp_database.py

## Construction of predictions file

In [1]:
# construct-predictions.py

## Construction of observations file

In [57]:
# construct_observations.py